In [1]:
%%time
%load_ext autoreload
%autoreload 2

Wall time: 52.9 ms


In [2]:
import numpy as np
import pandas as pd
import requests

In [9]:
date = 20200810
currencies_prices_url = f"https://www4.bcb.gov.br/Download/fechamento/{date}.csv"
currencies_info_url = f"https://www4.bcb.gov.br/Download/fechamento/{date}.csv"

cols = ['Timestamp', 'Cod Moeda', 'Tipo', 'Moeda', 'Taxa Compra', 'Taxa Venda', 'Paridade Compra', 'Paridade Venda'] 
df0 = pd.read_csv('20200810.csv', sep=";", names=cols) 
df0.head()

,Timestamp,Cod Moeda,Tipo,Moeda,Taxa Compra,Taxa Venda,Paridade Compra,Paridade Venda
0,10/08/2020,5,A,AFN,"0,07004000","0,07041000","76,60000000","77,00000000"
1,10/08/2020,9,A,ETB,"0,14930000","0,15220000","35,44690000","36,13150000"
2,10/08/2020,15,A,THB,"0,17330000","0,17340000","31,11000000","31,12000000"
3,10/08/2020,20,A,PAB,"5,39270000","5,39330000","1,00000000","1,00000000"
4,10/08/2020,27,A,VES,"0,00001940","0,00001940","277343,55110000","278038,64770000"
